<a href="https://colab.research.google.com/github/slnarayan/AddressEvaluationLLM/blob/main/AddressEvaluationUsingLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [2]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

llm_model = "gpt-3.5-turbo-0301"

def get_completion(prompt, model=llm_model):
  client = OpenAI( api_key = os.environ['OPENAI_API_KEY'],)
  messages = [{"role": "user", "content":prompt}]
  response = client.chat.completions.create(
      model=model,
      messages=messages,
      temperature=0.0,
  )
  return response.choices[0].message.content

In [3]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [4]:
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.6 MB/s eta 0:00:00


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.prompts import ChatPromptTemplate
import pandas as pd
import numpy as np

chat = ChatOpenAI(temperature=0.0, model=llm_model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [7]:
template_string = """Compare the two address that is given in text \
delimited by triple backticks and return \
'yes' if format matches otherwise return 'no' do not halucinate.\
text: ```{org}\n
{std}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

In [8]:
#Load the excel file
#loader = CSVLoader(file_path=file)
#data = loader.load()

#Format the zip column into string while loading the file
data_dict = { 'zip' : 'str', 'zip_s' : 'str'}

df = pd.read_csv('address.csv', dtype = data_dict, keep_default_na = False)
df = df.replace(np.nan, '')
df['status']= ''

In [9]:
#Format the rows in the excel containing individual original and standardized addresses
# into format to enable prompt

def get_addresses(df):
  #Format the original address
  l1 = df['line1']
  l2 = df['line2']
  city = df['city']
  state = df['state']
  zip = str(df['zip'])

  #Format the standardized address
  s_l1 = df['line1_s']
  s_l2 = df['line2_s']
  s_city = df['city_s']
  s_state = df['state_s']
  s_zip = str(df['zip_s'])

  org_addr = 'Valid Address: '+l1+', '+l2+', '+city+', '+state+', '+zip
  std_addr = 'Standardized Address: '+s_l1+', '+s_l2+', '+s_city+', '+s_state+', '+s_zip
  return org_addr, std_addr

In [10]:
#Iterate through the rows in the excel, create a prompt, call the llm, record the response.

for index, row in df.iterrows():
  org_addr, std_addr = get_addresses(row)
  prompt_messages = prompt_template.format_messages(
    org = org_addr,
    std = std_addr
  )
  # print(prompt_messages[0])

  # Call OpenAI llm
  chat_response = chat(prompt_messages)
  df['status'][index]= chat_response.content

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [11]:
df.to_csv('address_verified.csv', encoding='utf-8')
df

,line1,line2,city,state,zip,line1_s,line2_s,city_s,state_s,zip_s,status
0,10 Florence st,#201,malden,MA,02148,1018 Oleander Ln,,Waxhaw,NC,,no
1,1018 Oleander Ln,,Waxhaw,NC,,1018 Oleander Ln,,Waxhaw,NC,28173,yes
